This document details the techniques to generate missing data both on the theoretical aspect and on the code one. Plese refer to [produce_NA_example.ipynb](produce_NA_example.ipynb) for examples on how to generate data. 

References :
1. https://rmisstastic.netlify.app/how-to/python/generate_html/how%20to%20generate%20missing%20values
2. Stat Inference slide 71 (Davison)

## Notations and overview

Missing data are common in applications and real life datasets especially in Economics, Political and Social sciences either because some entities do not want to share some statistics or because they are not collected. Technical reasons could also cause missing data such as machine failures or non-responses. Missing data can significantly affect the conclusions drown from a dataset and uncertainty about the conclusions increases as the missing data increase.

We consider a data set $X \in \chi_1 \times \cdots \times \chi_p$ which is a concatenation of $d$ columns $X_j \in \chi_j$, where $\chi_j$ is the support of the variable $X_j$ which is of dimension $n$. This gives us a dataset of dimension $n \times d$. For example, one could have $\chi_j = \mathbb{R}^n, \mathbb{Z}^n, \mathcal{S}^n$, where $\mathcal{S} = {s_1, ...., s_k}$, for some quantitative or qualitative values $s_i$, $i=1..., k; k \in \mathbb{N}$

We now consider the response matrix $R\in \{0,1\}^{n \times d}$ defined by $R_{ij} = 1$ if $X_{ij}$ is observed and $0$ otherwise. We now partition the data matrix $X = \{X^{obs}, X^{miss}\}$, where $X^{obs}$ and $X^{miss}$ are the matrices containing the observed and missing values: $X^{obs}_{ij} = X_{ij} I_{\{R_{ij}=1\}}$, $X^{miss}_{ij} = X_{ij} I_{\{R_{ij}=0\}}$.

In the code, the matrix R is seen as a boolean tensor called a $\textit{mask}$ and has value True at position $(i,j)$, whenever $X_{ij}$, the i-th observation of the j-th variable is observed.

## The logistic model

To generate a response matrix $R$ or a mask in a non-independent way, one can use a logistic model with a sigmoid function. This consists in considering the sigmoid function 
$$\sigma(z) = \frac{1}{1 + e^{-z}}.$$
One will generate from a d-dimensional standard normal distribution weights $W$ and find a vector of intercepts $b \in \mathbb{R}^n$ such that 
$\sigma(WX + b) \in (0,1)^n$. We can then define the probability
$$\mathbb{P}(R_{ij} = 1 \mid X) = \sigma((WX)_i + b_i)$$
and generate a mask $R$ respecting these probabilities. Observe that for the missingness (R_{ij}) of the i-th observation of the variable $j$ depends on the other variables $X_{ij}$, $j=1,...,d$.


## Different types of missingness
Depending on the probability that an observation is missing

### MCAR - Missing Completely At Random
Observations are said to be be missing completely at random (MCAR) if $R \perp X$ that is if $\mathbb{P}(R \in A \mid X^{obs}, X^{miss}) = \mathbb{P}(R \in A)$ for any $A \in \sigma\{0,1\}^{n \times d}$.
To generate such missingness, one would start form a complete data matrix $X$ and generate an independent matrix $R$.

In the code, the mask corresponding to R is generated independently from a $n \times d$ uniform distribution and a given probabibility of observed values $p_{obs}$. With such mask, the data will have in expectation a proportion of missing data of $p_{miss} = 1 - p_{obs}$. Hence, here the missingness does not depend on the variables.

### MAR - Missing At Random
Observations are said to be be missing at random (MAR) if $R \perp X^{miss}$ that is if $\mathbb{P}(R \in A \mid X^{obs}, X^{miss}) = \mathbb{P}(R \in A \mid X^{obs})$ for any $A \in \sigma\{0,1\}^{n \times p}$.
To generate such missingness, one would start form a complete data matrix $X$ and generate a matrix $R$ independent form $X^{miss}$ but not from the observed data $X^{obs}$.

To generate such missingness, one needs again to generate a mask $R$ that depends on the observed values but not on the missing ones. To do so, we used a mask generated selecting at random (uniformly) $p_{obs} \cdot d$ variables (or columns) which will have no missing values. For the other variables, we use a logistic model to generate missingness. This then gives us a MAR response matrix.

### MNAR - Missing Not At Random
If missingness is not MCAR or MAR, it is said to be missing not at random (MNAR). To generate such data, one has various options. 

The first one, is to consider a self-masked model which will apply the logistic model to all variables, meaning that every variable will potentially have missing values (mecha == "MNAR", opt == "selfmasked").

The second one is to select a certain proportion p_params of variables which will be used as inputs for the logistic model and the remaining variables will have missing probabilities according to the logistic model. Then a MCAR mechanism is applied to the input variables. After this transformation one indeed has a dependence of the missingness of the two groups of variables and hence the mask $R$ depends on missing observations ($X^{miss}$) and on the observed ones ($X^{obs}$). This method is called using (mecha == "MNAR", opt == "logistic")

The third method is to use quantile censorship. TBC if we use quantile censorship
Use (mecha == "MNAR", opt == "quantile") with:
p_obs = proportion of variables with no missing values that will be used for the logistic masking model
q = quantile level at which cuts should occur.


### Generation of missing data
Logistic models are used to generate missing data for MAR and MNAR, one needs to provide the proportion of missing data

## Code applications

In [1]:
import numpy as np
from produce_NA import *

In [2]:
# Step 1: Generate complete data
# Produce np.ndarray with dim n x p
n = 100
d = 3
X = np.random.normal(size=(n, d))

p_miss_tot = 0.4 # total proportion of missing values
p_obs = 0.2 # proportion of totally observed variables (used for MAR and MNAR)
p_miss = p_miss_tot / (1-p_obs) # proportion of missing values for incomplete variables (used for MAR and MNAR)

In [6]:
#MCAR
X_mcar = produce_NA(X, p_miss=p_miss_tot, mecha='MCAR')['X_incomp']
mask_mcar = produce_NA(X, p_miss=p_miss_tot, mecha='MCAR')['mask']
#convert to np.ndarray
# Data with MCAR missingness: X_mcar
X_mcar = X_mcar.numpy()

In [7]:
#MAR (uses a logistic method to generate missingness on the (1-p_obs)*d variables that have missingness)
X_mar = produce_NA(X, p_miss=p_miss_tot, mecha='MAR', p_obs = p_obs)['X_incomp']

#convert to np.ndarray
# Data with MAR missingness: X_mar
X_mar = X_mar.numpy()

In [9]:
#MNAR + logistic (uses a logistic method to generate missingness on all d variables then applies a MCAR above it)
X_mnar_log = produce_NA(X, p_miss=p_miss_tot, mecha='MNAR', p_obs = p_obs, opt = 'logistic')['X_incomp']
X_mnar_log = X_mnar_log.numpy()

#MNAR + selfmask (uses a selfmask method to generate logistic missingness on all d variables then applies a MCAR above it)
X_mnar_selfmask = produce_NA(X, p_miss=p_miss_tot, mecha='MNAR', opt = 'selfmask')['X_incomp'] #equivalent to opt = 'logistic' and p_obs = 0
X_mnar_selfmask = X_mnar_selfmask.numpy()

#MNAR + quantile (uses a quantile method to generate missingness cutting after a certain threshold. NOTE: we did not use this method, it is just here for completeness)
q = 0.5 #quantile threshold
X_mnar_quantile = produce_NA(X, p_miss=p_miss, mecha='MNAR', opt = 'quantile', q = q, p_obs = p_obs)['X_incomp']
X_mnar_quantile = X_mnar_quantile.numpy()